# Loading our results 
This tutorial explains how to load the results that were published with our paper. The data can be found [here](https://doi.org/10.18126/unc8-336t). In this tutorial we'll use a snippet of that data, which contains a few tens of species that have experimental data. In a later tutorial we'll see how to load results that you run on your own molecules!

Let's first start with imports:

In [43]:
%load_ext autoreload
%autoreload 2

import pickle
import nglview as nv


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


There are three types of files that we can load. The file ending with `.json` contains detailed information about transition states (TSs), singlet-triplet crossings, etc., but does not contain any geometries. The two files ending with `.pickle` contain the most important summary data bout TSs and crossings, plus the actual geometries.

In the file ending with `_atoms.pickle`, the geometries are represented as [ASE](https://wiki.fysik.dtu.dk/ase/) atoms. In the file ending with `_xyz.pickle`, the geometries are given in the `xyz` format.

Let's load the file with ASE atoms objects:

In [44]:
with open("data/ts_query_atoms.pickle", 'rb') as f:
    info = pickle.load(f)


The keys in the dictionary are SMILES strings (we use the *cis* isomer for each species):

In [45]:
keys = list(info.keys())
key = keys[0]
print(key)

Fc1ccccc1/N=N\c1ccccc1F


Each value is a sub-dictionary with results:

In [46]:
sub_dic = info[key]
display(sub_dic.keys())

dict_keys(['results_by_mechanism', 'summary', 'stable', 'unstable'])

## `stable` and `unstable`

These keys tell you which isomer is thermodynamically stable, and which is unstable. Usually the unstable one is *cis* and that's also the case here:

In [47]:
display({"stable": sub_dic["stable"],
         "unstable": sub_dic["unstable"]})

{'stable': 'trans', 'unstable': 'cis'}

## `results_by_mechanism`

`results_by_mechanism` has results for each of the four mechanisms for each species. Let's see what it looks like:

In [36]:
display(sub_dic['results_by_mechanism'].keys())
display(sub_dic['results_by_mechanism']['cis'].keys())

dict_keys(['trans', 'cis'])

dict_keys(['s_t_crossing', 'ts'])

`results_by_mechanism` is divided into *cis* and *trans*. These provide activation free energies (and energies, enthalpies, etc.) relative to the *cis* and *trans* isomers, respectively. The classical activation free energies are in `ts`, while the effective activation free energies from intersystem crossing are in `s_t_crossing`.

### Classical transition states

Let's look at the classical transition states (TSs) first:

In [37]:
cis_dic = sub_dic['results_by_mechanism']['cis']
ts_list = cis_dic['ts']

print(type(ts_list))
print(len(ts_list))
display(list(ts_list[0].keys()))

<class 'list'>
4


['ts_geom_id',
 'endpoint_geom_id',
 'delta_free_energy',
 'delta_energy',
 'delta_enthalpy',
 'delta_conf_free_energy',
 'delta_avg_conf_energy',
 'delta_entropy_j_mol_k',
 'delta_conf_entropy_j_mol_k',
 'delta_vib_entropy_j_mol_k',
 'endpoint_conf_free_energy',
 'ts_conf_free_energy',
 'ts_atoms',
 'endpoint_atoms']

We see that `ts` is a list of 4 dictionaries, one for each mechanism. Here is the meaning of each key (ignore anything with `geom_id` in it):

- `delta_<quantity>`: Difference in `<quantity>` between the TS and the *cis* isomer (*cis* because we're looking at `sub_dic['results_by_mechanism']['cis']`). Different quantities are:
    - `energy`
    - `enthalpy`
    - `conf_free_energy`: Conformational free energy ($-TS_{\mathrm{conf}}$, where $S_{\mathrm{conf}}$ is the conformational entropy)
    - `entropy`
    - `conf_entropy`: Conformational entropy
    - `vib_entropy`: Vibrational entropy (entropy without the conformational compnonent)
    - `avg_conf_energy`: The average energy of the conformer ensemble, relative to the lowest energy conformer. Given in kcal/mol, and equal to $\sum_i p_i (E_i-E_{\mathrm{min}})$, where the sum is over conformers, $p_i = \mathrm{exp} (-E_i/k_{\mathrm{B}} T) \ / \ [\sum_j \mathrm{exp}(E_j/k_{\mathrm{B}} T)]$ is the probability of the $i^{\mathrm{th}}$ conformer, and $E_{\mathrm{min}}$ is the lowest energy in the ensemble.

- `endpoint_<quantity>`: the value of a quantity for an endpoint (i.e., reactant or product). Since we're look at `sub_dic['results_by_mechanism']['cis']`, the endpoint here is *cistrans*. If `quantity` is atoms, then the value is an ASE atoms object.

- `ts_<quantity>`: the value of a quantity for the TS

**Units**: By default, all *absolute* energies are given in Hartrees (atomic units), while all *relative* energies are given in kcal/mol. If `j_mol_k` is specified, then entropies are given in units of $\mathrm{J} / (\mathrm{mol \  K})$. Otherwise, both absolute and relative entropies are given in atomic units as $T \cdot S$, where $T=298.15 \ \mathrm{K}$ is the temprature. 

Let's look at each mechanism and visualize the TSs:



In [38]:
for ts_dic in ts_list:
    display({key: val for key, val in ts_dic.items() if 'geom_id' not in key
             and 'atoms' not in key})
    trj = [ts_dic[key] for key in ['endpoint_atoms', 'ts_atoms']]
    display(nv.show_asetraj(trj))

{'delta_free_energy': 30.820148961967195,
 'delta_energy': 32.984694537517385,
 'delta_enthalpy': 30.656765290672496,
 'delta_conf_free_energy': 0.18298129093378782,
 'delta_avg_conf_energy': 0.191860422567312,
 'delta_entropy_j_mol_k': -2.292796514160753,
 'delta_conf_entropy_j_mol_k': -2.5678139234176363,
 'delta_vib_entropy_j_mol_k': 0.2750174092568528,
 'endpoint_conf_free_energy': -0.001018021300492071,
 'ts_conf_free_energy': -0.0007264218319047075}

NGLWidget(max_frame=1)

{'delta_free_energy': 34.87174583758515,
 'delta_energy': 36.367195562381816,
 'delta_enthalpy': 34.98519002647618,
 'delta_conf_free_energy': -0.05169594959487424,
 'delta_avg_conf_energy': 0.23552907112562413,
 'delta_entropy_j_mol_k': 1.5919855318466523,
 'delta_conf_entropy_j_mol_k': 0.7254598460672609,
 'delta_vib_entropy_j_mol_k': 0.8665256857793607,
 'endpoint_conf_free_energy': -0.001018021300492071,
 'ts_conf_free_energy': -0.0011004041023241948}

NGLWidget(max_frame=1)

{'delta_free_energy': 34.87431903986289,
 'delta_energy': 36.36717288290034,
 'delta_enthalpy': 34.98258057858952,
 'delta_conf_free_energy': -0.046874688701910416,
 'delta_avg_conf_energy': 0.23437111005318326,
 'delta_entropy_j_mol_k': 1.5192563408761184,
 'delta_conf_entropy_j_mol_k': 0.6578021047418857,
 'delta_vib_entropy_j_mol_k': 0.861454236134204,
 'endpoint_conf_free_energy': -0.001018021300492071,
 'ts_conf_free_energy': -0.0010927209282295383}

NGLWidget(max_frame=1)

{'delta_free_energy': 30.81704745307957,
 'delta_energy': 32.98490621267779,
 'delta_enthalpy': 30.660028493818967,
 'delta_conf_free_energy': 0.1804201662241379,
 'delta_avg_conf_energy': 0.19201172595300736,
 'delta_entropy_j_mol_k': -2.2034792069304268,
 'delta_conf_entropy_j_mol_k': -2.5318731359443003,
 'delta_vib_entropy_j_mol_k': 0.3283939290138611,
 'endpoint_conf_free_energy': -0.001018021300492071,
 'ts_conf_free_energy': -0.0007305032470073593}

NGLWidget(max_frame=1)

We see two inversion mechanisms and two rotations. The rotational barrier is 4 kcal/mol lower than the inversion barrier.

### Singlet-triplet crossings

Now let's look at singlet-triplet crossings:

In [39]:
s_t_list = cis_dic['s_t_crossing']
display(s_t_list)


[[{'s_t_crossing_geom_id': 243495721,
   'endpoint_geom_id': 223213660,
   'delta_free_energy': 26.481164145500973,
   'delta_enthalpy': 26.317780474206263,
   'delta_conf_free_energy': 0.18298129093378782,
   'delta_avg_conf_energy': 0.191860422567312,
   'delta_eff_free_energy': 28.90471414781525,
   'delta_entropy_j_mol_k': -2.292796514160753,
   'delta_conf_entropy_j_mol_k': -2.5678139234176363,
   'delta_vib_entropy_j_mol_k': 0.2750174092568528,
   'delta_eff_entropy_j_mol_k': -36.30296994928712,
   'endpoint_conf_free_energy': -0.001018021300492071,
   's_t_crossing_conf_free_energy': -0.0007264218319047075,
   'endpoint': 'cis',
   't_isc': 5.83526486568076e-12,
   's_t_crossing_atoms': Atoms(symbols='FC6N2C6FH8', pbc=False),
   'endpoint_atoms': Atoms(symbols='FC6N2C6FH8', pbc=False)},
  {'s_t_crossing_geom_id': 243495723,
   'endpoint_geom_id': 248437562,
   'delta_free_energy': 25.63805796424636,
   'delta_enthalpy': 25.47467429295165,
   'delta_conf_free_energy': 0.182981290

Many of the keys are the same as for TSs, but the structure is different. Here we have a list of lists. Each sub-list has two dictionaries, one for the crossing closer to *cis*, and one for the crosisng closer to *trans*. The closer isomer is indicated by `endpoint`. We can test to see if that's true:

In [40]:
trj = [d['s_t_crossing_atoms'] for d in s_t_list[0]]
display(nv.show_asetraj(trj))
diheds = [i.get_dihedral(6, 7, 8, 9) for i in trj]
print(diheds)


NGLWidget(max_frame=1)

[75.19895318677847, 105.40228347744296]


As promised, the first geometry is closer to *cis* (the first dictionary has `'endpoint': 'cis'`), and the second is closer to *trans*. We can see that both visually and from looking at the dihedral angles, which are $75^{\circ} < 90^{\circ} $ and $105^{\circ} > 90^{\circ}$, respectively.

There are also some keys that aren't in the TS dictionary:

- `delta_eff_free_energy`: The effective free energy difference between the crossing geometry and the higher energy relaxed isomer (in this case, the *cis* isomer). It is given by

$\Delta G^{X, \mathrm{eff}} = \Delta H^{\mathrm{X}} + \Delta S^{\mathrm{app}}$

$ \Delta S^{\mathrm{app}} = \Delta S^{\mathrm{X}} + k_{\mathrm{B}} \left( \mathrm{log} \frac{h k_{\mathrm{ISC}} }{k_{\mathrm{B}} T } - \frac{1}{2} \right)  $


 Here $\Delta H^{\mathrm{X}}$ is the difference in enthalpy between the crossing point ("`X`") and the endpoint. $\Delta S^{\mathrm{app}}$ is the entropy that would be inferred from experiment if one assumed an Arrhenius TS relation. It includes the actual entropy difference, $\Delta S^{\mathrm{X}}$, and a term related to the intersystem crossing rate, $k_{\mathrm{ISC}}$.

- `delta_eff_entropy`: the effective (or apparent) activation entropy defined above 
- `s_t_crossing_conf_free_energy`: the conformational free energy at the crossing point
- `t_isc`: the intersystem crossing time, defined as $1/k_{\mathrm{ISC}}$, where $k_{\mathrm{ISC}}$ is the intersystem crossing rate. Given in seconds.
    - Note that `t_isc` is about 1.3 times longer for the crossing on the *cis* side than the crossing on the *trans* side. The rate is therefore 1.3 times smaller. This is because the rate gets multiplied by 2 when going from singlet to triplet (*cis* side), and three when going from triplet to singlet (*trans* side). The ratio is not exactly 1.5, since the different potential energy surfaces on either side also affect the rates.



## `summary` 

Now that we've seen `results_by_mechanism`, let's take a look at `summary`:

In [63]:
summary = sub_dic["summary"]
display(summary.keys())


dict_keys(['s_t_crossing', 'trans', 'cis'])

`summary` has three keys:
- `trans`: Information about the TS free energy, energy, etc. relative to the *trans* isomer
- `cis`: Information about the TS free energy, energy, etc. relative to the *cis* isomer
- `s_t_crossing`: summary information about singlet-triplet crossing


###  `trans`  and `cis`
`summary["trans"]` and `summary["cis"]` look a lot like one of our mechanism dictionaries, but it's specifically for the TS with the lowest free energy:

In [64]:
display(list(summary["trans"].keys()))
print(summary["trans"].keys() == summary["cis"].keys())

['ts_geom_id',
 'endpoint_geom_id',
 'delta_free_energy',
 'delta_energy',
 'delta_enthalpy',
 'delta_conf_free_energy',
 'delta_avg_conf_energy',
 'delta_entropy_j_mol_k',
 'delta_conf_entropy_j_mol_k',
 'delta_vib_entropy_j_mol_k',
 'endpoint_conf_free_energy',
 'ts_conf_free_energy',
 'ts_atoms',
 'endpoint_atoms']

True


### `s_t_crossing`.

Now let's look at `s_t_crossing`:

In [68]:
display(list(summary["s_t_crossing"].keys()))
display(list(summary["s_t_crossing"]["unstable_side"].keys()))
print(summary["s_t_crossing"]["unstable_side"].keys() == summary["s_t_crossing"]["stable_side"].keys())

['unstable_side', 'stable_side']

['s_t_crossing_geom_id',
 'endpoint_geom_id',
 'delta_free_energy',
 'delta_enthalpy',
 'delta_conf_free_energy',
 'delta_avg_conf_energy',
 'delta_eff_free_energy',
 'delta_entropy_j_mol_k',
 'delta_conf_entropy_j_mol_k',
 'delta_vib_entropy_j_mol_k',
 'delta_eff_entropy_j_mol_k',
 'endpoint_conf_free_energy',
 's_t_crossing_conf_free_energy',
 'endpoint',
 't_isc',
 's_t_crossing_atoms',
 'endpoint_atoms']

True


We see that `s_t_crossing` has two separate dictionaries, one with free energies (and effective free energies, enthalpies, etc.) relative to the unstable isomer, and one relative to the stable isomer.

In [75]:
for key, dic in summary["s_t_crossing"].items():
    print(key)
    display({key: val for key, val in dic.items() if 'geom_id' not in key
             and 'atoms' not in key})
    trj = [dic[key] for key in ['endpoint_atoms', 's_t_crossing_atoms']]
    display(nv.show_asetraj(trj))

unstable_side


{'delta_free_energy': 26.481164145500973,
 'delta_enthalpy': 26.317780474206263,
 'delta_conf_free_energy': 0.18298129093378782,
 'delta_avg_conf_energy': 0.191860422567312,
 'delta_eff_free_energy': 28.90471414781525,
 'delta_entropy_j_mol_k': -2.292796514160753,
 'delta_conf_entropy_j_mol_k': -2.5678139234176363,
 'delta_vib_entropy_j_mol_k': 0.2750174092568528,
 'delta_eff_entropy_j_mol_k': -36.30296994928712,
 'endpoint_conf_free_energy': -0.001018021300492071,
 's_t_crossing_conf_free_energy': -0.0007264218319047075,
 'endpoint': 'cis',
 't_isc': 5.83526486568076e-12}

NGLWidget(max_frame=1)

stable_side


{'delta_free_energy': 25.63805796424636,
 'delta_enthalpy': 25.47467429295165,
 'delta_conf_free_energy': 0.18298129093378782,
 'delta_avg_conf_energy': 0.191860422567312,
 'delta_eff_free_energy': 27.898200365791066,
 'delta_entropy_j_mol_k': -2.292796514160753,
 'delta_conf_entropy_j_mol_k': -2.5678139234176363,
 'delta_vib_entropy_j_mol_k': 0.2750174092568528,
 'delta_eff_entropy_j_mol_k': -34.00983762790571,
 'endpoint_conf_free_energy': -0.001018021300492071,
 's_t_crossing_conf_free_energy': -0.0007264218319047075,
 'endpoint': 'trans',
 't_isc': 4.428758216975692e-12}

NGLWidget(max_frame=1)

# Conclusion

In this tutorial we learned how to load and understand the published barrier data. Now you're ready for the next tutorial, in which we plot and analyze the data!